<a href="https://colab.research.google.com/github/roydendsouza31/AI-Generated-Images-vs-Real-Images/blob/R3/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AI Generated Images vs Real Images

In [2]:
! pip install simple_image_download
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2 as cv
from simple_image_download import simple_image_download as simp
import random
import PIL
import pickle

In [3]:
response = simp.simple_image_download
limit = 100

In [4]:
response().download("ai generated art", 100)
response().download("ai generated images", 100)
response().download("ai generated characters", 100)
response().download('stable diffusion', 100)
response().download('dalle2 generated images', 100)
response().download('midjourney', 100)
response().download('cityscapes', 100)
response().download('animals', 100)
response().download('vehicles', 50)
response().download('traffic', 50)
response().download('offices', 30)
response().download('real food images', 50)
response().download('landscape', 13)

HTTPConnectionPool(host='no.access', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7c962c5cdd50>: Failed to resolve 'no.access' ([Errno -2] Name or service not known)"))


In [70]:
! mkdir -p /content/dataset_train/AIGenerated
! mkdir -p /content/dataset_train/Real
! mkdir -p /content/dataset_test/AIGenerated
! mkdir -p /content/dataset_test/Real

In [17]:
!mv /content/simple_images/ai\ generated\ art/* /content/dataset_train/ai_generated
!mv /content/simple_images/ai\ generated\ characters/* /content/dataset_train/ai_generated
!mv /content/simple_images/ai\ generated\ images/* /content/dataset_train/ai_generated
!mv /content/simple_images/dalle2\ generated\ images/* /content/dataset_train/ai_generated
!mv /content/simple_images/midjourney/* /content/dataset_train/ai_generated
!mv /content/simple_images/stable\ diffusion/* /content/dataset_train/ai_generated
!mv /content/simple_images/animals/* /content/dataset_train/real
!mv /content/simple_images/cityscapes/* /content/dataset_train/real
!mv /content/simple_images/landscape/* /content/dataset_train/real
!mv /content/simple_images/offices/* /content/dataset_train/real
!mv /content/simple_images/real\ food\ images/* /content/dataset_train/real
!mv /content/simple_images/traffic/* /content/dataset_train/real
!mv /content/simple_images/vehicles/* /content/dataset_train/real

In [19]:
! rm -r /content/simple_images
! rm -r sample_data

In [23]:
data = "/content/dataset_train"
categories = ['Real', 'AIGenerated']

In [42]:
import os

aigenerated_folder = "/content/dataset_train/AIGenerated"
for filename in os.listdir(aigenerated_folder):
    if ' ' in filename:
        os.rename(os.path.join(aigenerated_folder, filename), os.path.join(aigenerated_folder, filename.replace(' ', '_')))


In [41]:
import os

real_folder = "/content/dataset_train/Real"
for filename in os.listdir(real_folder):
    if ' ' in filename:
        os.rename(os.path.join(real_folder, filename), os.path.join(real_folder, filename.replace(' ', '_')))


In [43]:
img_size = 48
training_data = []

In [50]:
for category in categories:
    path = os.path.join(data, category)
    classes = categories.index(category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        img_array = cv.imread(img_path)
        if img_array is not None:
            img_resized = cv.resize(img_array, (img_size, img_size))
            img_resized = img_resized / 255.0  # Normalize pixel values to [0, 1]
            training_data.append([img_resized, classes])
            print("Success")
        else:
            print(f"Failed to load image: \"{img_path}\"")

print(f"Total images processed: {len(training_data)}")

Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Failed to load image: "/content/dataset_train/Real/cityscapes_16.jpg"
Success
Success
Failed to load image: "/content/dataset_train/Real/vehicles_30.jpg"
Success
Success
Success
Success
Success
Success
Failed to load image: "/content/dataset_train/Real/traffic_35.jpg"
Failed to load image: "/content/dataset_train/Real/real_food_images_48.jpg"
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Failed to load image: "/content/dataset_train/Real/cityscapes_77.jpg"
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Failed to load image: "/content/dataset_train/Real/cityscapes_13.

In [51]:
len(training_data), training_data[0][0].shape

(1055, (48, 48, 3))

In [52]:
random.shuffle(training_data)

In [53]:
X_train = []
y_train = []

for features, label in training_data:
    X_train.append(features)
    y_train.append(label)

X_train = np.array(X_train).reshape(-1, img_size, img_size, 3)
y_train = np.array(y_train)

In [54]:
X_train.shape

(1055, 48, 48, 3)

In [55]:
y_train.shape

(1055,)

In [56]:
X_train[0].shape

(48, 48, 3)

In [57]:
pickle_out = open("X_train.pickle", "wb")
pickle.dump(X_train, pickle_out, protocol=4)
pickle_out.close()

pickle_out = open("y_train.pickle", "wb")
pickle.dump(y_train, pickle_out, protocol=4)
pickle_out.close()

In [58]:
pickle_in = open("X_train.pickle", "rb")
X_train = pickle.load(pickle_in)

pickle_in = open("y_train.pickle", "rb")
y_train = pickle.load(pickle_in)

In [59]:
X_train.shape

(1055, 48, 48, 3)

In [60]:
y_train.shape

(1055,)

In [61]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Sequential

In [62]:
model = keras.Sequential([
    keras.layers.Conv2D(32,(3,3), activation='relu', input_shape = (48,48,3)),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Dropout(0.2),

    keras.layers.Conv2D(64,(3,3), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Dropout(0.2),

    keras.layers.Conv2D(128,(3,3), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Dropout(0.2),

    keras.layers.Conv2D(256,(3,3), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Dropout(0.2),

    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [63]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 23, 23, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 64)        0

In [64]:
model.fit(X_train, y_train, epochs=15)

Epoch 1/15
33/33 [==============================] - 8s 21ms/step - loss: 0.6846 - accuracy: 0.5289
Epoch 2/15
33/33 [==============================] - 0s 7ms/step - loss: 0.6861 - accuracy: 0.5299
Epoch 3/15
33/33 [==============================] - 0s 7ms/step - loss: 0.6685 - accuracy: 0.6000
Epoch 4/15
33/33 [==============================] - 0s 7ms/step - loss: 0.6418 - accuracy: 0.6730
Epoch 5/15
33/33 [==============================] - 0s 7ms/step - loss: 0.6163 - accuracy: 0.6654
Epoch 6/15
33/33 [==============================] - 0s 7ms/step - loss: 0.5818 - accuracy: 0.7062
Epoch 7/15
33/33 [==============================] - 0s 7ms/step - loss: 0.5473 - accuracy: 0.7441
Epoch 8/15
33/33 [==============================] - 0s 7ms/step - loss: 0.5421 - accuracy: 0.7479
Epoch 9/15
33/33 [==============================] - 0s 7ms/step - loss: 0.5130 - accuracy: 0.7507
Epoch 10/15
33/33 [==============================] - 0s 7ms/step - loss: 0.4968 - accuracy: 0.7545
Epoch 11/15
33/33 

In [65]:
model.save("AIGeneratedModel.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [66]:
model_new = keras.models.load_model("AIGeneratedModel.h5")

In [69]:
data = "content/dataset_test/"
categories = ['Real', 'AIGenerated']

img_size = 48
testing_data = []

i = 0
for category in categories:
    path = os.path.join(data,category)
    classes = categories.index(category)
    for img in os.listdir(path):
        i = i + 1
        img_array = cv.imread(os.path.join(path,img))
        new_array = cv.resize(img_array, (48,48))
        new_array = new_array/255
        testing_data.append([new_array, classes])

random.shuffle(testing_data)

X_test = []
y_test = []

for features, label in testing_data:
    X_test.append(features)
    y_test.append(label)

X_test = np.array(X_test).reshape(-1, img_size, img_size, 3)
y_test = np.array(y_test)

FileNotFoundError: [Errno 2] No such file or directory: 'content/dataset_test/Real'